In [ ]:
!pip install tenacity

In [ ]:
import geopandas
from carbonplan_retro.data import cat

ecomap_sections = geopandas.read_file(
    "/home/jovyan/lost+found/cleland-ecosections/sections"
).to_crs("epsg:4326")

In [ ]:
from carbonplan_retro.arbitrage.project_arbitrage import get_crs

from mpl_toolkits.axes_grid1.inset_locator import inset_axes


from matplotlib.colors import LinearSegmentedColormap

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
from carbonplan_retro.data import cat

retro_json = cat.retro_db_light_json.read()

import matplotlib.pyplot as plt

from shapely.geometry import Point

In [ ]:
from carbonplan_retro.arbitrage.prism_arbitrage import load_prism_arbitrage

arbitrage_df = load_prism_arbitrage(79)

In [ ]:
ecosections = ["M261A", "M261B", "M261D"]

In [ ]:
proj_centroids = {
    project["opr_id"]: Point(project["shape_centroid"][0][0], project["shape_centroid"][0][1])
    for project in retro_json
    if 79 in project["supersection_ids"]
}

In [ ]:
proj_points = geopandas.GeoDataFrame(
    data=list(proj_centroids.keys()), geometry=list(proj_centroids.values()), crs="epsg:4326"
)

In [ ]:
from carbonplan_retro.load.geometry import load_states

states = load_states()
conus = [
    "AL",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DE",
    "FL",
    "GA",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY",
]
states = states.loc[states["postal"].isin(conus)]
outline_states = states.dissolve("adm0_a3")

sub_states = states.loc[states["postal"].isin(["CA", "OR"])]

In [ ]:
crs = get_crs()

In [ ]:
arbitrage_df.mean_local_slag.mean()

In [ ]:
vmin, vmax = arbitrage_df.mean_local_slag.quantile(0.025), arbitrage_df.mean_local_slag.quantile(
    0.975
)

norm = mpl.colors.Normalize(vmin, vmax)

In [ ]:
vmin, vmax

In [ ]:
rfia_mean = 116.91
slag_per_section = {
    "M261B": 148.86 - rfia_mean,
    "M261A": 122.65 - rfia_mean,
    "M261D": 91.84 - rfia_mean,
}

In [ ]:
def make_colormap():
    N = 256
    vals1 = np.ones((N, 4))
    vals1[:, 0] = np.linspace(126 / 256, 1, N)
    vals1[:, 1] = np.linspace(179 / 256, 1, N)
    vals1[:, 2] = np.linspace(106 / 256, 1, N)
    vals2 = np.ones((N, 4))
    vals2[:, 0] = np.linspace(128 / 256, 1, N)
    vals2[:, 1] = np.linspace(128 / 256, 1, N)
    vals2[:, 2] = np.linspace(128 / 256, 1, N)
    return mpl.colors.ListedColormap(np.concatenate([vals2, vals1[::-1]]))

In [ ]:
cp_cm = make_colormap()
ecomap_colors = cp_cm(norm(list(slag_per_section.values())))
ecomap_cmap = ListedColormap(ecomap_colors[::-1], name="ecomap")

In [ ]:
supersection_sections = (
    ecomap_sections.loc[ecomap_sections["MAP_UNIT_S"].isin(ecosections)].to_crs(crs).copy()
)
supersection_sections["slag"] = supersection_sections["MAP_UNIT_S"].map(slag_per_section)
supersection_outline = supersection_sections.dissolve("PROJECT")

In [ ]:
supersection_sections

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7, 7))


left, bottom, width, height = 0.7, 0.2, 0.3, 0.05
cax = fig.add_axes([left, bottom, width, height])


arbitrage_df.to_crs(crs).plot(
    "delta_slag",
    markersize=4.5,
    ax=ax[0],
    cmap=cp_cm,
    legend=True,
    cax=cax,
    vmin=-55,
    vmax=55,
    legend_kwds={"label": "Aboveground Carbon", "orientation": "horizontal"},
)

proj_points.to_crs(crs).plot(ax=ax[0], marker="^", color=".3", markersize=100)
supersection_outline.plot(ax=ax[0], edgecolor="k", lw=0.2, color="None")

supersection_sections.plot(
    column="slag",
    ax=ax[1],
    edgecolor="k",
    lw=0.2,
    cmap=cp_cm,
    vmin=-55,
    vmax=55,
)
proj_points.to_crs(crs).plot(ax=ax[1], marker="^", color=".3", markersize=100)

width = 4.5
inset = inset_axes(
    ax[1],
    width=width,
    height=width / 2,
    bbox_to_anchor=(1.8, 0.65, 0.55, 0.275),
    bbox_transform=ax[1].transAxes,
)
outline_states.to_crs(crs).plot(ax=inset, color="None", edgecolor=".8")
supersection_sections.to_crs(crs).plot(ax=inset, color=".1", alpha=0.55)
inset.axes.set_axis_off()

inset_2 = inset_axes(
    ax[1],
    width=4 / 3,
    height=4,
    bbox_to_anchor=(1.1, 0.4, 0.75, 0.25),
    bbox_transform=ax[1].transAxes,
)
sub_states.to_crs(crs).plot(ax=inset_2, color="None", edgecolor=".8")
supersection_sections.to_crs(crs).plot(ax=inset_2, color=".1", alpha=0.55)


inset_2.axes.set_axis_off()


ax[0].set_axis_off()
ax[1].set_axis_off()
# [spine.set_visible(False) for spine in ax[0].spines.values()]
# [spine.set_visible(False) for spine in ax[1].spines.values()]
fig.tight_layout()
plt.savefig("/home/jovyan/pub-figs/southern-cascades-arbitrage.pdf", dpi=300, bbox_inches="tight")